<font size=6><b> lec04. 평가 검증

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action="ignore")


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV

# Glass Data Load

* ref : https://archive.ics.uci.edu/ml/datasets/glass+identification


Attribute Information:
<pre>
1. Id number: 1 to 214
2. RI: refractive index
3. Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
4. Mg: Magnesium
5. Al: Aluminum
6. Si: Silicon
7. K: Potassium
8. Ca: Calcium
9. Ba: Barium
10. Fe: Iron
11. Type of glass: (class attribute)
-- 1 building_windows_float_processed
-- 2 building_windows_non_float_processed
-- 3 vehicle_windows_float_processed
-- 4 vehicle_windows_non_float_processed (none in this database)
-- 5 containers
-- 6 tableware
-- 7 headlamps



In [2]:
glass = pd.read_csv("./dataset/iris.csv")

## [Quiz] 데이터프레임 구조 확인

In [3]:
glass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sl      150 non-null    float64
 1   sw      150 non-null    float64
 2   pl      150 non-null    float64
 3   pw      150 non-null    float64
 4   target  150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


## [Quiz] 결측처리
* 피쳐 k  : 결측 '0'으로 채우기
* 피쳐 fe : 결측 '평균값'으로 채우기, 평균값은 반올림해 소수점2자리까지만 사용
* 반올림 : round(), 올림 : math.ceil(), 내림:math.floor(), 소수점 버림:math.trunc() 

In [4]:
glass.isna().sum()[glass.isna().sum() > 0]

Series([], dtype: int64)

* 피쳐 k : 결측 '0'으로 채우기

glass['k'].fillna(0, inplace=True)
glass['k'] = glass['k'].fillna(0)

* 피쳐 fe : 결측  glass_type별 fe '평균값'으로 채우기, 평균값은 반올림해 소수점2자리까지만 사용 

glass["fe"] = glass["fe"].fillna(   glass.groupby("glass_type")["fe"].transform("mean")    )

## [Quiz] 신규(파생) 피쳐 만들기
* 'household' 피쳐 추가
* Window 용 glass type의 경우 0의 값, 그 외(Household용)  glass type의 경우 1의 값  

<pre>
11. Type of glass: (class attribute)
-- 1 building_windows_float_processed
-- 2 building_windows_non_float_processed
-- 3 vehicle_windows_float_processed
-- 4 vehicle_windows_non_float_processed (none in this database)
-- 5 containers
-- 6 tableware
-- 7 headlamps

glass['household'] = glass['glass_type'].map({1:0, 2:0, 3:0, 4:0,5:1, 6:1, 7:1})
glass.head()

# 모델 학습 및 평가

## [Quiz] train_test 분리
* test 데이터 크기는 전체 데이터의 20% 사용

In [5]:

y = glass['target']
X = glass.drop(['target'],  axis=1)
print(X.shape, y.shape)

(150, 4) (150,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((120, 4), (30, 4), (120,), (30,))

## [Quiz] LogisticRegression 모델을 사용해 f1 점수 출력
* 분류모델 : from sklearn.linear_model import LogisticRegression
* 모델은 기본 파라미터 사용

* "기본 피쳐 + 파생변수"  모델 점수

In [7]:
lr_model = LogisticRegression(penalty='l2', C=1.0)
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

# print(f'intercept   = {lr_model.intercept_}')
# print(f'coefficient = {lr_model.coef_}')

In [8]:


print('Confusion matrix')
print(confusion_matrix(y_test, lr_pred))

print(f"Precision : {precision_score(y_test, lr_pred, average='macro'):.4f}")
print(f"Recall    : {recall_score(y_test, lr_pred, average='macro'):.4f}")
print(f"F1        : {f1_score(y_test, lr_pred, average='macro'):.4f}")
print(f"Accuracy  : {accuracy_score(y_test, lr_pred):.4f}")

Confusion matrix
[[11  0  0]
 [ 0 12  1]
 [ 0  0  6]]
Precision : 0.9524
Recall    : 0.9744
F1        : 0.9610
Accuracy  : 0.9667


## 데이터학습량 늘리기
* sklearn.model_selection.KFold             무작위 증강
* sklearn.model_selection.StratifiedKFold   target을 train,test에 골로루 증강
* sklearn.model_selection.cross_val_score   증강해서 점수만 리턴
* sklearn.model_selection.GridSearchCV      증강 + 모델튜닝

In [9]:
# k = KFold()
# sk = StratifiedKFold()
# gcv = GridSearchCV

# cross_val_score()

# KFold
* class sklearn.model_selection.KFold(n_splits=5, *, shuffle=False, random_state=None)

In [10]:
# 시계열은 shuffle 놉! 섞어버리면 안되지 shuffle = False 무 족 권 굳이 할거면 random_state 넣으렴 
# KFold로 돌렸다? 데이터 들어오는게 다 달라! 그래서 여러번 학습하는 효과를 누릴 수 있어 적은 데이터로도! 
# split 5하세요? 원본을 다섯배 증강시켜주세여! 즉 5배 학습효과를 늘려주겠다! 는 뜻 ! 


# KFold(n_splits=2, random_state=None, shuffle=False)   덩어리를 2개로 나뉘어서 train, test로 나눠주기 
# >>> for train_index, test_index in kf.split(X):



# df.iloc[줄,칸]

#  X_train, X_test = X[train_index], X[test_index]
# ...     y_train, y_test = y[train_index], y[test_index]   이렇게 한 번 도는거고! 
# 이제 이걸 5번 돌린다는거야! 

In [11]:
k = KFold(n_splits= 3 , shuffle=True , random_state=1111)  # shuffle막으니까 0점만 나오자나 
# print(k.get_n_splits(glass))
#-----------------------------train_test_split
f1_list = []
for train_index, test_index in k.split(glass):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index] , X.iloc[test_index]
    y_train, y_test = y.iloc[train_index] , y.iloc[test_index]
    # print( X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    #---------------------- 학습해라! (웬만해서는 여기서 학습하진 말고. . . . .)
    lr_model = LogisticRegression(penalty='l2', C=1.0)
    lr_model.fit(X_train, y_train)
    computer_pred  = lr_model.predict(X_test)
    
    f1 = f1_score( y_test, computer_pred, average = 'macro')
    f1_list.append(f1)
    print(f1)
    print(confusion_matrix(y_test, computer_pred))
    
print(np.array(f1_list).mean()) # f1의 평균점수! 

TRAIN: [  0   2   5   7   8   9  11  13  14  15  17  18  20  21  22  23  25  26
  29  30  31  32  35  37  38  39  40  42  43  44  45  47  48  50  52  53
  54  55  57  61  62  63  64  66  68  69  73  74  75  76  77  78  81  83
  85  86  87  88  89  91  93  94  96  97  98 102 104 105 106 107 108 109
 110 112 114 115 117 118 121 123 124 125 127 129 130 132 133 134 135 136
 137 139 140 141 143 144 146 147 148 149] TEST: [  1   3   4   6  10  12  16  19  24  27  28  33  34  36  41  46  49  51
  56  58  59  60  65  67  70  71  72  79  80  82  84  90  92  95  99 100
 101 103 111 113 116 119 120 122 126 128 131 138 142 145]
0.9592592592592593
[[17  0  0]
 [ 0 17  1]
 [ 0  1 14]]
TRAIN: [  0   1   3   4   6   8   9  10  11  12  14  16  18  19  20  21  24  25
  27  28  29  31  33  34  36  41  42  44  45  46  47  49  50  51  52  53
  55  56  58  59  60  61  64  65  67  69  70  71  72  75  76  77  79  80
  81  82  84  85  86  87  88  90  92  93  95  96  97  98  99 100 101 103
 104 105 108 109 111 

# StratifiedKFold
* class sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)

In [12]:
# skf = StratifiedKFold(n_splits=2)
# >>> skf.get_n_splits(X, y)
# 2
# >>> print(skf)
# StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
# >>> for train_index, test_index in skf.split(X, y):   >>> train , test 분리해줘! 왜? 비율나눠줘야하니까 
# ...     print("TRAIN:", train_index, "TEST:", test_index)
# ...     X_train, X_test = X[train_index], X[test_index]
# ...     y_train, y_test = y[train_index], y[test_index]

In [13]:
skf = StratifiedKFold(n_splits=2, shuffle = True , random_state = 1111)  # y값을 각 fold안에 골고루 넣어주기 때문에 셔플 굳이 열 필요없어! 

for train_index ,test_index  in skf.split(X  , y):
    train_X, test_X = X.iloc[train_index], X.iloc[test_index]
    train_y , test_y = y.iloc[train_index], y.iloc[test_index]
    print("TRAIN:", train_index, "TEST:", test_index)
    print(train_X.shape, test_X.shape , train_y.shape , test_y.shape )
    print(X_train.value_counts())
    print(y_train.value_counts())
    
    lr_model = LogisticRegression(penalty='l2', C=1.0)
    lr_model.fit(X_train, y_train)
    computer_pred  = lr_model.predict(X_test)
    
    f1 = f1_score( y_test, computer_pred, average = 'macro')
    f1_list.append(f1)
    print(f1)
    print(confusion_matrix(y_test, computer_pred))
    
print(np.array(f1_list).mean()) # f1의 평균점수! 

TRAIN: [  0   3   4   6  12  15  16  17  18  19  21  22  26  28  31  32  33  34
  35  36  37  38  45  48  49  51  53  54  56  60  61  65  66  70  71  72
  73  74  75  81  82  84  88  90  91  92  93  96  97  98 101 103 104 109
 111 112 113 115 116 118 121 124 125 127 128 130 133 136 137 138 139 142
 145 146 149] TEST: [  1   2   5   7   8   9  10  11  13  14  20  23  24  25  27  29  30  39
  40  41  42  43  44  46  47  50  52  55  57  58  59  62  63  64  67  68
  69  76  77  78  79  80  83  85  86  87  89  94  95  99 100 102 105 106
 107 108 110 114 117 119 120 122 123 126 129 131 132 134 135 140 141 143
 144 147 148]
(75, 4) (75, 4) (75,) (75,)
sl   sw   pl   pw 
5.8  2.7  5.1  1.9    2
4.3  3.0  1.1  0.1    1
6.1  2.9  4.7  1.4    1
6.3  3.3  4.7  1.6    1
     2.9  5.6  1.8    1
                     ..
5.2  3.5  1.5  0.2    1
     3.4  1.4  0.2    1
     2.7  3.9  1.4    1
5.1  3.8  1.6  0.2    1
7.9  3.8  6.4  2.0    1
Length: 99, dtype: int64
0    35
2    34
1    31
Name: target, d

# cross_val_score()
* sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)

In [14]:
scores_arr = cross_val_score(lr_model , X, y , scoring="f1_macro"  # ndarray로 리턴해줄게!  
                , cv=3)

print(scores_arr)
print(scores_arr.mean())

# for train_index ,test_index  in skf.split(X  , y):     이걸 전부 cross_val_score가 다 알아서 처리해주고 결과만 리턴해줌! 
#     train_X, test_X = X.iloc[train_index], X.iloc[test_index]
#     train_y , test_y = y.iloc[train_index], y.iloc[test_index]
#     print("TRAIN:", train_index, "TEST:", test_index)
#     print(train_X.shape, test_X.shape , train_y.shape , test_y.shape )
#     print(X_train.value_counts())
#     print(y_train.value_counts())
                                                                # 중간과정의 개입이 안들어가짐.. 비교, 가공하는 등 처리해야 될 것들이 있다? 그럼 다른거 쓰렴
                                                                # 딱히 별 것 없으면 그냥 cross_val 쓰렴! 
#     lr_model = LogisticRegression(penalty='l2', C=1.0)
#     lr_model.fit(X_train, y_train)
#     computer_pred  = lr_model.predict(X_test)
    
#     f1 = f1_score( y_test, computer_pred, average = 'macro')
#     f1_list.append(f1)
#     print(f1)
#     print(confusion_matrix(y_test, computer_pred))
      # return np.array(f1_list)

[0.9797235  0.95925926 0.98037518]
0.9731193139795291


# 모델교체 

<pre>
* CART 모델 >>>> 회귀랑 분류 다 돼 
tree: 가장 기본적! 
    트리의 가장 대표적인 모델? decisiontreeclassifier
    가장 큰 단점? 과적합이여 . ..(있는 데이터에 딱 맞춰서 학습시킨거! >> 새로운 유형의 데이터가 나오면 또로로록...) 
    
bagging: 
    가장 대표적 모델? randomforestclassifier
boosting: 데이터 계속 복제해 >> 틀린쪽 버리고 잘맞춘쪽 가중치 쎄게 줘! 즉 가중치 조절해서 좀 더 선명하고 좋은 가지 ㄱㄱ,, (잔챙이나 의미없는 데이터 가중치조절_버렷) >> 이쁜가지로 최종 FITTING함 >>> 다양한 모델을 쓴 효과를 얻을 수 있어! 
    가장 대표적 모델? adaboostclassifier
                     xgboost lightgbm catboost (과적합을 피함 // 피쳐가 너무 많아 )

votingclassifier: 조아 배깅과 부스팅 섞어! ! ! !! 
stackingclassifier

* ML에서 부트스트랩은 복원 추출을 얘기함!
  

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from  sklearn.ensemble import GradientBoostingClassifier


In [16]:
model_list = [ ("LR" , LogisticRegression(penalty='l2', C=1.0)),
               ("DT", DecisionTreeClassifier()),
               ("RF",RandomForestClassifier()), # DecisionTreeClassifier 100개 돌린 효과 有
               ("GB",GradientBoostingClassifier())
             ]
for (name,model) in model_list:
    print(name)
    scores_arr = cross_val_score(model , X , y , scoring="f1_macro",cv=5)  # ndarray로 리턴해줄게!  
    # print(scores_arr.mean())

LR
DT
RF
GB


<pre>
모델학습에 상당히 큰 영향을 미치는 파라미터?? 하이퍼 파라미터
oob_score = true하면 따로 test 줄 필요없어 그냥 나머지 데이터로 알아서 해쥼 

gini 불순도 
불순도 높은 애들을 가지치기 하는거여 
entropy 정보량
정보량이 많은 쪽으로 이동한다! 

* 그리드씨브이의 장점? 증강도 증강인데 ,, , , 파라미터를 손댐으로써 모델을 교체하지 않더라도 모델튜닝이 가능하다! ! ! 

# GridSearchCV()

* class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

* RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

* GridSearchCV(estimator, param_grid, , scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2n_jobs', error_score=nan, return_train_score=False)

In [23]:
rf_model = RandomForestClassifier(n_estimators=100,random_state=1111)
# X = np.concatenate((X_train_scaler, X_test_scaler), axis=0)
# y = np.concatenate((y_train, y_test), axis=0)

myprm = {
             "criterion"         : ["gini", "entropy"]
            , "max_depth"         : [5,7,9,11]
            , "min_samples_split" : [1,2,3] 
        }
fold = StratifiedKFold(n_splits=5) #shuffle=True, random_state=1111                 
gcv_model = GridSearchCV(rf_model
                         , param_grid=myprm
                         , scoring="f1_macro"
                         , refit=True
                         , cv = fold        
                           #cv=5
                       ) 
gcv_model.fit(X, y)  #내가 설정한대로 학습해보렴!  ! ! !
print(gcv_model.best_score_)
print(gcv_model.best_params_)

# gcv이용해서 가장 좋은 파라미터를 가지고 있는 모델을 찾아따 ?! 

# cv , , , 회귀는 kfold가 디폴트야! 회귀는연속성의 데이터가 많아서 골고루의 개념이 업어
#분류는  skfold가 디폴트임 

0.9664818612187034
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 3}


* 결측이없다
* 모든 피쳐타잉베 oobject가 없다!


# 5차 : 평가검증
* 내일할거
* f1, recall , accuracy , precision , roc_curv, auc
* confusion_matrix
* precision , recall : threadhold
